In [3]:
import numpy as np
import scipy.signal as signal
from pydub import AudioSegment

def morse_code_translator(signal, sample_rate):
    # Apply FFT
    fft_result = np.fft.fft(signal)
    freqs = np.fft.fftfreq(len(signal), 1 / sample_rate)
    
    # Find important frequencies
    # Example: You might determine important frequencies based on their magnitudes in the FFT result
    
    # Filter design
    # Example: You might design a bandpass filter based on the important frequencies
    
    # Apply bandpass filter
    # Example: You might use scipy's bandpass filter function
    # filtered_signal = signal.lfilter(b, a, signal)
    
    return filtered_signal



c:\Users\Logan\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [10]:
def main(file_path):
    # Read audio file
    audio = AudioSegment.from_file(file_path)
    sample_rate = audio.frame_rate
    print(sample_rate)
    signal_data = np.array(audio.get_array_of_samples())
    print(signal_data)

    # Apply morse code translator
    filtered_signal = morse_code_translator(signal_data, sample_rate)

    # Output filtered signal or further process it
    
if __name__ == "__main__":
    file_path = "Morse_code_message_hello_everyone.mp3"  # Replace with your MP3 file path
    main(file_path)


c:\Users\Logan\AppData\Local\Programs\Python\Python310\lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)


FileNotFoundError: [WinError 2] The system cannot find the file specified